In [ ]:
# 드라이브 마운트 및 기본 설정
from google.colab import drive
drive.mount('/content/drive')

import os
import sys
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 경로 설정
BASE_DIR = '/content/drive/MyDrive/Colab Notebooks/woke-odds'
TASK1_PATH = os.path.join(BASE_DIR, 'test_classifier.txt')
TASK2_PATH = os.path.join(BASE_DIR, 'final_tgt_test.txt')

In [ ]:
# 두 번째 셀: Task1 데이터 로드 및 확인
def load_task1_data(file_path):
    """
    Task1 데이터 로드 (test_classifier.txt)

    Returns:
        list: 각 샘플이 딕셔너리 형태로 저장된 리스트
    """
    data = []

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:
                item = json.loads(line)
                data.append(item)

    return data

# Task1 데이터 로드
task1_data = load_task1_data(TASK1_PATH)

print(f"Task1 데이터 개수: {len(task1_data)}")
print(f"\n=== Task1 샘플 데이터 ===")
print(json.dumps(task1_data[0], indent=2, ensure_ascii=False))

# 레이블 분포 확인
label_counts = {}
for item in task1_data:
    label = item['label']
    label_counts[label] = label_counts.get(label, 0) + 1

print(f"\n=== 레이블 분포 ===")
print(f"모호함 (label=1): {label_counts.get('1', 0)}개")
print(f"명확함 (label=0): {label_counts.get('0', 0)}개")

Task1 데이터 개수: 1175

=== Task1 샘플 데이터 ===
{
  "entity2": "Going Back to Cali <S> music.composition media_common.cataloged_instance media_common.creative_work ratings.rated_entity <S> Going Back To Cali is a song recorded by rapper The Notorious B.I.G. and Puff Daddy from the album Life After Death. The song is noted as one of Biggie's popular songs from the album, well known for its catchy hook and beat. The song contains a sample of Zapp hit More Bounce to the Ounce.",
  "entity1": "Going Back to Cali <S> ratings.rated_entity commerce.consumer_product media_common.cataloged_instance media_common.creative_work music.album commerce.product <S> Going Back to Cali is a 1988 single by LL Cool J.",
  "label": "0",
  "context": "what is the musical composition that was recorded on Going Back to Cali?"
}

=== 레이블 분포 ===
모호함 (label=1): 503개
명확함 (label=0): 672개


In [ ]:
# Task2 데이터 로드 및 확인
def load_task2_data(file_path):
    """
    Task2 데이터 로드 (final_tgt_test.txt)

    Returns:
        list: 각 줄의 명확화 질문이 저장된 리스트
    """
    data = []

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:
                data.append(line)

    return data

# Task2 데이터 로드
task2_data = load_task2_data(TASK2_PATH)

print(f"Task2 데이터 개수: {len(task2_data)}")
print(f"\n=== Task2 샘플 데이터 (첫 5개) ===")
for i, clarification in enumerate(task2_data[:5]):
    print(f"{i+1}. {clarification}")

Task2 데이터 개수: 503

=== Task2 샘플 데이터 (첫 5개) ===
1. Which one do you mean, brand Aldara or non brand Aldara, when you say the active constituent?
2. Which one do you mean, non nominated work The Comet or nominated work The Comet, when you say the dress designer?
3. Which one do you mean, non winning work The End of Summer or winning work The End of Summer, when you say the producing firm?
4. When you say the variant of the release of the album, are you talking about winning work Hello Dolly or non winning work Hello Dolly?
5. Are you talking about non nominated work Believer or nominated work Believer, when you say the album that contains believer?


In [ ]:
# 데이터 정렬 및 통합
def align_and_merge_data(task1_data, task2_data):
    """
    Task1과 Task2 데이터를 정렬하고 통합

    Args:
        task1_data: Task1 데이터 (모든 쿼리 + 레이블)
        task2_data: Task2 데이터 (모호한 쿼리의 명확화 질문만)

    Returns:
        list: 통합된 데이터셋
    """
    # Task1에서 모호한 쿼리만 필터링
    ambiguous_queries = [item for item in task1_data if item['label'] == '1']
    clear_queries = [item for item in task1_data if item['label'] == '0']

    print(f"모호한 쿼리 수: {len(ambiguous_queries)}")
    print(f"명확한 쿼리 수: {len(clear_queries)}")
    print(f"Task2 명확화 질문 수: {len(task2_data)}")

    # 데이터 개수 검증
    if len(ambiguous_queries) != len(task2_data):
        print(f"\n⚠️ 경고: 모호한 쿼리 수({len(ambiguous_queries)})와 "
              f"Task2 데이터 수({len(task2_data)})가 일치하지 않습니다!")
        print("두 데이터셋의 순서가 정렬되어 있는지 확인이 필요합니다.")

    # 통합 데이터셋 생성
    integrated_data = []

    # 1. 모호한 쿼리 처리 (Task2와 매칭)
    for i, item in enumerate(ambiguous_queries):
        processed_item = {
            'query': item['context'],
            'is_ambiguous': 1,  # 1 = 모호함
            'gold_clarification': task2_data[i] if i < len(task2_data) else None
        }
        integrated_data.append(processed_item)

    # 2. 명확한 쿼리 처리
    for i, item in enumerate(clear_queries):
        processed_item = {
            'query': item['context'],
            'is_ambiguous': 0  # 0 = 명확함
        }
        integrated_data.append(processed_item)

    return integrated_data

# 데이터 정렬 및 통합
integrated_data = align_and_merge_data(task1_data, task2_data)

print(f"\n=== 통합 완료 ===")
print(f"총 데이터 수: {len(integrated_data)}")
print(f"모호한 쿼리: {sum(1 for item in integrated_data if item['is_ambiguous'] == 1)}개")
print(f"명확한 쿼리: {sum(1 for item in integrated_data if item['is_ambiguous'] == 0)}개")

모호한 쿼리 수: 503
명확한 쿼리 수: 672
Task2 명확화 질문 수: 503

=== 통합 완료 ===
총 데이터 수: 1175
모호한 쿼리: 503개
명확한 쿼리: 672개


In [ ]:
# 통합 데이터 확인 및 검증
print("=== 모호한 쿼리 샘플 ===")
ambiguous_sample = [item for item in integrated_data if item['is_ambiguous'] == 1][0]
print(json.dumps(ambiguous_sample, indent=2, ensure_ascii=False))

print("\n=== 명확한 쿼리 샘플 ===")
clear_sample = [item for item in integrated_data if item['is_ambiguous'] == 0][0]
print(json.dumps(clear_sample, indent=2, ensure_ascii=False))

# 데이터 무결성 검증
print("\n=== 데이터 무결성 검증 ===")

# 1. 모호한 쿼리에 gold_clarification이 있는지 확인
ambiguous_data = [item for item in integrated_data if item['is_ambiguous'] == 1]
missing_clarification = sum(1 for item in ambiguous_data if item.get('gold_clarification') is None)
print(f"Gold clarification 누락된 모호한 쿼리: {missing_clarification}개")

# 2. 명확한 쿼리에 gold_clarification 필드가 있는지 확인
clear_data = [item for item in integrated_data if item['is_ambiguous'] == 0]
has_clarification = sum(1 for item in clear_data if 'gold_clarification' in item)
print(f"Gold clarification 필드가 있는 명확한 쿼리: {has_clarification}개 (0이어야 정상)")

# 3. query 필드 확인
empty_queries = sum(1 for item in integrated_data if not item.get('query', '').strip())
print(f"빈 쿼리: {empty_queries}개")

# 4. 전체 통계
total = len(integrated_data)
ambiguous_count = len(ambiguous_data)
clear_count = len(clear_data)
print(f"\n총 데이터 수: {total}")
print(f"  - 모호한 쿼리 (is_ambiguous=1): {ambiguous_count}개")
print(f"  - 명확한 쿼리 (is_ambiguous=0): {clear_count}개")

print("\n 데이터 전처리 완료")

=== 모호한 쿼리 샘플 ===
{
  "query": "What is Aldara's ingredient",
  "is_ambiguous": 1,
  "gold_clarification": "Which one do you mean, brand Aldara or non brand Aldara, when you say the active constituent?"
}

=== 명확한 쿼리 샘플 ===
{
  "query": "what is the musical composition that was recorded on Going Back to Cali?",
  "is_ambiguous": 0
}

=== 데이터 무결성 검증 ===
Gold clarification 누락된 모호한 쿼리: 0개
Gold clarification 필드가 있는 명확한 쿼리: 0개 (0이어야 정상)
빈 쿼리: 0개

총 데이터 수: 1175
  - 모호한 쿼리 (is_ambiguous=1): 503개
  - 명확한 쿼리 (is_ambiguous=0): 672개

 데이터 전처리 완료


In [ ]:
# 모호한 쿼리 샘플 5개 확인
print("=== 모호한 쿼리 샘플 5개 ===\n")
ambiguous_samples = [item for item in integrated_data if item['is_ambiguous'] == 1][:5]

for i, sample in enumerate(ambiguous_samples, 1):
    print(f"[샘플 {i}]")
    print(f"Query: {sample['query']}")
    print(f"Gold Clarification: {sample['gold_clarification']}")
    print("-" * 80)
    print()

=== 모호한 쿼리 샘플 5개 ===

[샘플 1]
Query: What is Aldara's ingredient
Gold Clarification: Which one do you mean, brand Aldara or non brand Aldara, when you say the active constituent?
--------------------------------------------------------------------------------

[샘플 2]
Query: The Comet's costume degisners
Gold Clarification: Which one do you mean, non nominated work The Comet or nominated work The Comet, when you say the dress designer?
--------------------------------------------------------------------------------

[샘플 3]
Query: What is production company of The End of Summer
Gold Clarification: Which one do you mean, non winning work The End of Summer or winning work The End of Summer, when you say the producing firm?
--------------------------------------------------------------------------------

[샘플 4]
Query: State the type of release of Hello Dolly
Gold Clarification: When you say the variant of the release of the album, are you talking about winning work Hello Dolly or non winning

In [ ]:
# 통합된 데이터셋을 DataFrame으로 변환해서 확인
import pandas as pd

# DataFrame 생성
df = pd.DataFrame(integrated_data)

print("=== 통합 데이터셋 기본 정보 ===")
print(f"총 데이터 개수: {len(df)}")
print(f"\n컬럼 목록: {df.columns.tolist()}")
print(f"\nis_ambiguous 분포:")
print(df['is_ambiguous'].value_counts().sort_index())

print("\n=== 모호한 쿼리 샘플 (head 5개) ===")
ambiguous_df = df[df['is_ambiguous'] == 1].head()
print(ambiguous_df.to_string(index=False))

print("\n=== 명확한 쿼리 샘플 (head 5개) ===")
clear_df = df[df['is_ambiguous'] == 0].head()
print(clear_df.to_string(index=False))

# 데이터 구조 요약
print("\n=== 데이터 구조 요약 ===")
print(f"모호한 쿼리 (is_ambiguous=1): {len(df[df['is_ambiguous']==1])}개")
print(f"  - 필드: query, is_ambiguous, gold_clarification")
print(f"\n명확한 쿼리 (is_ambiguous=0): {len(df[df['is_ambiguous']==0])}개")
print(f"  - 필드: query, is_ambiguous")

=== 통합 데이터셋 기본 정보 ===
총 데이터 개수: 1175

컬럼 목록: ['query', 'is_ambiguous', 'gold_clarification']

is_ambiguous 분포:
is_ambiguous
0    672
1    503
Name: count, dtype: int64

=== 모호한 쿼리 샘플 (head 5개) ===
                                          query  is_ambiguous                                                                                                                    gold_clarification
                    What is Aldara's ingredient             1                                         Which one do you mean, brand Aldara or non brand Aldara, when you say the active constituent?
                  The Comet's costume degisners             1                     Which one do you mean, non nominated work The Comet or nominated work The Comet, when you say the dress designer?
What is production company of The End of Summer             1         Which one do you mean, non winning work The End of Summer or winning work The End of Summer, when you say the producing firm?
       State the ty

In [ ]:
# 전처리된 데이터 저장
# DataFrame으로 변환
df = pd.DataFrame(integrated_data)

# 저장 경로 설정
output_dir = os.path.join(BASE_DIR, 'evaluation_data')
os.makedirs(output_dir, exist_ok=True)

output_path = os.path.join(output_dir, 'claqua_integrated_test.json')
csv_output_path = os.path.join(output_dir, 'claqua_integrated_test.csv')

# JSON 형식으로 저장
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(integrated_data, f, ensure_ascii=False, indent=2)

# CSV 형식으로도 저장 (확인용)
df.to_csv(csv_output_path, index=False, encoding='utf-8-sig')

print(f"전처리된 데이터 저장 완료!")
print(f"JSON: {output_path}")
print(f"CSV: {csv_output_path}")
print(f"\n총 {len(integrated_data)}개의 샘플이 저장되었습니다.")

전처리된 데이터 저장 완료!
JSON: /content/drive/MyDrive/Colab Notebooks/woke-odds/evaluation_data/claqua_integrated_test.json
CSV: /content/drive/MyDrive/Colab Notebooks/woke-odds/evaluation_data/claqua_integrated_test.csv

총 1175개의 샘플이 저장되었습니다.


In [ ]:
# 일곱 번째 셀: 통계 정보 출력
print("=== CLAQUA 평가 데이터셋 통계 ===\n")

# 기본 통계
total_count = len(integrated_data)
ambiguous_count = sum(1 for item in integrated_data if item['is_ambiguous'])
clear_count = total_count - ambiguous_count

print(f"총 샘플 수: {total_count}")
print(f"  - 모호한 쿼리: {ambiguous_count} ({ambiguous_count/total_count*100:.1f}%)")
print(f"  - 명확한 쿼리: {clear_count} ({clear_count/total_count*100:.1f}%)")

# 쿼리 길이 통계
query_lengths = [len(item['query'].split()) for item in integrated_data]
print(f"\n쿼리 길이 통계 (단어 수):")
print(f"  - 평균: {np.mean(query_lengths):.1f}")
print(f"  - 최소: {np.min(query_lengths)}")
print(f"  - 최대: {np.max(query_lengths)}")
print(f"  - 중앙값: {np.median(query_lengths):.1f}")

# 명확화 질문 길이 통계 (모호한 쿼리만)
clarification_lengths = [
    len(item['gold_clarification'].split())
    for item in integrated_data
    if item['is_ambiguous'] and item['gold_clarification']
]
if clarification_lengths:
    print(f"\n명확화 질문 길이 통계 (단어 수):")
    print(f"  - 평균: {np.mean(clarification_lengths):.1f}")
    print(f"  - 최소: {np.min(clarification_lengths)}")
    print(f"  - 최대: {np.max(clarification_lengths)}")
    print(f"  - 중앙값: {np.median(clarification_lengths):.1f}")

print("\n 전처리 완료 - 이제 모델 추론 및 평가를 진행할 수 있습니다.")

=== CLAQUA 평가 데이터셋 통계 ===

총 샘플 수: 1175
  - 모호한 쿼리: 503 (42.8%)
  - 명확한 쿼리: 672 (57.2%)

쿼리 길이 통계 (단어 수):
  - 평균: 7.1
  - 최소: 2
  - 최대: 18
  - 중앙값: 7.0

명확화 질문 길이 통계 (단어 수):
  - 평균: 23.3
  - 최소: 11
  - 최대: 51
  - 중앙값: 23.0

 전처리 완료 - 이제 모델 추론 및 평가를 진행할 수 있습니다.
